https://github.com/seg/2016-ml-contest/blob/master/Bird_Team/Facies_classification_7.ipynb

In [ ]:
#matplotlib inline
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict, LeaveOneGroupOut, LeavePGroupsOut
from sklearn.metrics import confusion_matrix, make_scorer, f1_score, accuracy_score, recall_score, precision_score
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel, RFECV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.base import clone
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
pd.options.mode.chained_assignment = None

In [ ]:
filename = 'facies_PE_ET.csv'
training_data = pd.read_csv(filename)
print(set(training_data["Well Name"]))
training_data.head()

{'LUKE G U', 'SHANKLE', 'CHURCHMAN BIBLE', 'Recruit F9', 'NEWBY', 'ALEXANDER D', 'SHRIMPLIN', 'CROSS H CATTLE', 'KIMZEY A', 'NOLAN'}


,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,3,A1 SH,SHRIMPLIN,2793.0,77.45,0.664,9.9,11.915,4.6,1,1.000
1,3,A1 SH,SHRIMPLIN,2793.5,78.26,0.661,14.2,12.565,4.1,1,0.979
2,3,A1 SH,SHRIMPLIN,2794.0,79.05,0.658,14.8,13.050,3.6,1,0.957
3,3,A1 SH,SHRIMPLIN,2794.5,86.10,0.655,13.9,13.115,3.5,1,0.936
4,3,A1 SH,SHRIMPLIN,2795.0,74.58,0.647,13.5,13.300,3.4,1,0.915


In [ ]:
well_data = pd.read_csv('validation_data_nofacies.csv')
print(set(well_data["Well Name"]))
print(well_data.shape)
well_data.head()

{'STUART', 'CRAWFORD'}
(800, 10)


,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,A1 SH,STUART,2808.0,66.276,0.630,3.3,10.65,3.591,1,1.000
1,A1 SH,STUART,2808.5,77.252,0.585,6.5,11.95,3.341,1,0.978
2,A1 SH,STUART,2809.0,82.899,0.566,9.4,13.60,3.064,1,0.956
3,A1 SH,STUART,2809.5,80.671,0.593,9.5,13.25,2.977,1,0.933
4,A1 SH,STUART,2810.0,75.971,0.638,8.7,12.35,3.020,1,0.911


In [ ]:
well_data["origin"] = 'test'
training_data["origin"] = 'train'
df = pd.concat([well_data,training_data],axis=0,ignore_index=True)[list(training_data.columns)]
df['Well Name'] = df['Well Name'].astype('category')
df.head(10)

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,origin
0,NaN,A1 SH,STUART,2808.0,66.276,0.630,3.3,10.65,3.591,1,1.000,test
1,NaN,A1 SH,STUART,2808.5,77.252,0.585,6.5,11.95,3.341,1,0.978,test
2,NaN,A1 SH,STUART,2809.0,82.899,0.566,9.4,13.60,3.064,1,0.956,test
3,NaN,A1 SH,STUART,2809.5,80.671,0.593,9.5,13.25,2.977,1,0.933,test
4,NaN,A1 SH,STUART,2810.0,75.971,0.638,8.7,12.35,3.020,1,0.911,test
5,NaN,A1 SH,STUART,2810.5,73.955,0.667,6.9,12.25,3.086,1,0.889,test
6,NaN,A1 SH,STUART,2811.0,77.962,0.674,6.5,12.45,3.092,1,0.867,test
7,NaN,A1 SH,STUART,2811.5,83.894,0.667,6.3,12.65,3.123,1,0.844,test
8,NaN,A1 SH,STUART,2812.0,84.424,0.653,6.7,13.05,3.121,1,0.822,test
9,NaN,A1 SH,STUART,2812.5,83.160,0.642,7.3,12.95,3.127,1,0.800,test


In [ ]:
# add some features based on the well data. 

# nb points : can be correlated with how soft soil is ? 
print("session")
sessionsize = df.groupby(["Well Name",'Formation']).size().reset_index()
sessionsize.columns =  ["Well Name",'Formation','formation_size']
df = pd.merge(df,sessionsize,how='left',on = ["Well Name",'Formation'])

# depth : 
print("depth")
sessionsize = df.groupby(["Well Name",'Formation'])["Depth"].min().reset_index()
sessionsize.columns =  ["Well Name",'Formation','minimum_depth']
df = pd.merge(df,sessionsize,how='left',on = ["Well Name",'Formation'])

sessionsize = df.groupby(["Well Name",'Formation'])["Depth"].max().reset_index()
sessionsize.columns =  ["Well Name",'Formation','maximum_depth']
df = pd.merge(df,sessionsize,how='left',on = ["Well Name",'Formation'])

df['formation_depth'] = df["maximum_depth"] - df["minimum_depth"]

df["soft_indic"] = df['formation_depth'] / df["formation_size"]

# add avgs of feat
print("add avgs of feat")
list_to_avg = ['Depth', 'GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
for val in list_to_avg : 
    df[val + "_min"] = df.groupby(["Well Name",'Formation'])[val].transform(np.min)
    df[val + "_max"] = df.groupby(["Well Name",'Formation'])[val].transform(np.max)
    df[val + "_mean"] = df.groupby(["Well Name",'Formation'])[val].transform(np.mean)
    df[val + "_var"] = df.groupby(["Well Name",'Formation'])[val].transform(np.var) 

# add distances feat. = an attempt at regulariation.
print("add distances feat.")
for val in list_to_avg : 
    df[val + "_min_dist"] = df[val] -df[val + "_min"]
    df[val + "_max_dist"] =  df[val] -df[val + "_max"]
    df[val + "_mean_dist"] =  df[val] -df[val + "_mean"]
    
# add lag and lead !
print("lag lead")
list_to_lag = ['Depth', 'GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
for val in list_to_lag:
    for lag in range(1,11):
        df[val+'_lag_'+str(lag)]=df[val]-df.groupby("Well Name")[val].shift(periods=lag)
        df[val+'_lead_'+str(lag)]=df[val]-df.groupby("Well Name")[val].shift(periods=-lag)

# adding some Formation lag and lead. 
for lag in range(1,3):
    df['Formation'+'_lag_'+str(lag)]=df.groupby("Well Name")['Formation'].shift(periods=lag)
    df['Formation'+'_lead_'+str(lag)]=df.groupby("Well Name")['Formation'].shift(periods=-lag)
    df['Formation'+'_lag_'+str(lag) + 'equal'] = (df['Formation'+'_lag_'+str(lag)] == df["Formation"]).astype(int)
    df['Formation'+'_lead_'+str(lag) + 'equal'] = (df['Formation'+'_lead_'+str(lag)] == df["Formation"]).astype(int) 

print("rolling")
#Add rolling features
list_to_roll = ['Depth', 'GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M','RELPOS']
window_size = [5,10,15,20,50]
for w in window_size:
    for val in list_to_roll:
        df[val+'_rollingmean_'+str(w)]=df.groupby("Well Name")[val].apply(
            lambda x:x.rolling(window=w,center=True).mean())
        df[val+'_rollingmax_'+str(w)]=df.groupby("Well Name")[val].apply(
            lambda x:x.rolling(window=w,center=True).max())
        df[val+'_rollingmin_'+str(w)]=df.groupby("Well Name")[val].apply(
            lambda x:x.rolling(window=w,center=True).min())
        df[val+'_rollingstd_'+str(w)]=df.groupby("Well Name")[val].apply(
            lambda x:x.rolling(window=w,center=True).std())
        
print("special window features for NM_M")
def NM_M_distance(x,how,target):
    length = len(x)
    rank = np.empty(length)
    count = -1
    NM_M = x["NM_M"].values
    if how=="up":
        order = range(length)
    elif how=="down":
        order = range(length-1,-1,-1)
    for i in order:
        if ((NM_M[i] != target) & (count>-1)):
            count+=1
            rank[i] += count
        elif NM_M[i] == target:
            count=0
        else:
            rank[i] = count
    rank = pd.DataFrame(rank.astype(int), columns=["NM_M_Rank_Target_+"+str(target)+"_"+how], index = x.index)
    return(rank)
df["NM_M_Rank_Target_1_up"]=df.groupby(["Well Name"]).apply(NM_M_distance,how="up",target=1)
df["NM_M_Rank_Target_2_up"]=df.groupby(["Well Name"]).apply(NM_M_distance,how="up",target=2)
df["NM_M_Rank_Target_1_down"]=df.groupby(["Well Name"]).apply(NM_M_distance,how="down",target=1)
df["NM_M_Rank_Target_2_down"]=df.groupby(["Well Name"]).apply(NM_M_distance,how="down",target=2)

print("filling na")
df = df.groupby(["Well Name"], as_index=False).apply(lambda group: group.bfill())
df = df.groupby(["Well Name"], as_index=False).apply(lambda group: group.ffill())
df = df.fillna(df.mean())

print("Vectorizing Formation text data")
from sklearn.feature_extraction.text import CountVectorizer
list_formation = ['Formation',
 'Formation_lag_1',
 'Formation_lead_1',
 'Formation_lag_2',
 'Formation_lead_2']
for l in list_formation:
    cv = CountVectorizer()
    counts = cv.fit_transform(df[l].values)
    cols = [c+"_"+l for c in cv.get_feature_names()]
    counts = pd.DataFrame(counts.toarray(),columns = cols)
    df = df.drop(l,axis = 1)
    df = pd.concat([df,counts],axis=1)

print("Finished preparing data. Now ready for ML ignition!")


session
depth
add avgs of feat
add distances feat.
lag lead
rolling
special window features for NM_M
filling na
Vectorizing Formation text data
Finished preparing data. Now ready for ML ignition!


In [ ]:
df

,Facies,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,origin,formation_size,minimum_depth,maximum_depth,formation_depth,soft_indic,Depth_min,Depth_max,Depth_mean,Depth_var,GR_min,GR_max,GR_mean,GR_var,ILD_log10_min,ILD_log10_max,ILD_log10_mean,ILD_log10_var,DeltaPHI_min,DeltaPHI_max,DeltaPHI_mean,DeltaPHI_var,PHIND_min,PHIND_max,PHIND_mean,PHIND_var,PE_min,PE_max,PE_mean,PE_var,...,a1_Formation,b1_Formation,b2_Formation,b3_Formation,b4_Formation,b5_Formation,lm_Formation,sh_Formation,a1_Formation_lag_1,b1_Formation_lag_1,b2_Formation_lag_1,b3_Formation_lag_1,b4_Formation_lag_1,b5_Formation_lag_1,lm_Formation_lag_1,sh_Formation_lag_1,a1_Formation_lead_1,b1_Formation_lead_1,b2_Formation_lead_1,b3_Formation_lead_1,b4_Formation_lead_1,b5_Formation_lead_1,lm_Formation_lead_1,sh_Formation_lead_1,a1_Formation_lag_2,b1_Formation_lag_2,b2_Formation_lag_2,b3_Formation_lag_2,b4_Formation_lag_2,b5_Formation_lag_2,lm_Formation_lag_2,sh_Formation_lag_2,a1_Formation_lead_2,b1_Formation_lead_2,b2_Formation_lead_2,b3_Formation_lead_2,b4_Formation_lead_2,b5_Formation_lead_2,lm_Formation_lead_2,sh_Formation_lead_2
0,4.503254,STUART,2808.0,66.276,0.630,3.300,10.650,3.591,1,1.000,test,41,2808.0,2829.0,21.0,0.512195,2946.0,2973.0,2960.221154,58.209936,65.180,149.650,73.312500,128.064313,0.56,0.711,0.619308,0.000668,-18.300,10.700,-0.438462,46.044374,13.99,55.915,22.391442,65.800813,1.800,3.60,2.917308,0.135185,...,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1
1,4.503254,STUART,2808.5,77.252,0.585,6.500,11.950,3.341,1,0.978,test,41,2808.0,2829.0,21.0,0.512195,2946.0,2973.0,2960.221154,58.209936,65.180,149.650,73.312500,128.064313,0.56,0.711,0.619308,0.000668,-18.300,10.700,-0.438462,46.044374,13.99,55.915,22.391442,65.800813,1.800,3.60,2.917308,0.135185,...,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1
2,4.503254,STUART,2809.0,82.899,0.566,9.400,13.600,3.064,1,0.956,test,41,2808.0,2829.0,21.0,0.512195,2946.0,2973.0,2960.221154,58.209936,65.180,149.650,73.312500,128.064313,0.56,0.711,0.619308,0.000668,-18.300,10.700,-0.438462,46.044374,13.99,55.915,22.391442,65.800813,1.800,3.60,2.917308,0.135185,...,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1
3,4.503254,STUART,2809.5,80.671,0.593,9.500,13.250,2.977,1,0.933,test,41,2808.0,2829.0,21.0,0.512195,2946.0,2973.0,2960.221154,58.209936,65.180,149.650,73.312500,128.064313,0.56,0.711,0.619308,0.000668,-18.300,10.700,-0.438462,46.044374,13.99,55.915,22.391442,65.800813,1.800,3.60,2.917308,0.135185,...,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1
4,4.503254,STUART,2810.0,75.971,0.638,8.700,12.350,3.020,1,0.911,test,41,2808.0,2829.0,21.0,0.512195,2946.0,2973.0,2960.221154,58.209936,65.180,149.650,73.312500,128.064313,0.56,0.711,0.619308,0.000668,-18.300,10.700,-0.438462,46.044374,13.99,55.915,22.391442,65.800813,1.800,3.60,2.917308,0.135185,...,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4944,5.000000,CHURCHMAN BIBLE,3120.5,46.719,0.947,1.828,7.254,3.617,2,0.685,train,44,3101.0,3122.5,21.5,0.488636,3101.0,3122.5,3111.750000,41.250000,14.469,58.313,38.841773,128.680955,0.71,1.294,1.029500,0.011631,-21.832,4.296,0.432205,23.711891,2.32,47.721,7.583432,64.829416,1.605,5.02,3.979409,0.478129,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4945,5.000000,CHURCHMAN BIBLE,3121.0,44.563,0.953,2.241,8.013,3.344,2,0.677,train,44,3101.0,3122.5,21.5,0.488636,3101.0,3122.5,3111.750000,41.250000,14.469,58.313,38.841773,128.680955,0.71,1.294,1.029500,0.011631,-21.832,4.296,0.432205,23.71

In [ ]:
# this time let's use all the training set 
groups = df[(df['origin']=='train')]["Well Name"]
ytrain = df[(df['origin']=='train')]['Facies']
yvalid = df[(df['origin']=='test')]['Facies']
xtrain = df[(df['origin']=='train')].drop(['Well Name','origin','Facies'],axis=1)
xvalid = df[(df['origin']=='test')].drop(['Well Name','origin','Facies'],axis=1)
custom_cv = LeaveOneGroupOut()

In [ ]:
clf_rfe = RandomForestClassifier(
    n_estimators=100,
    criterion="entropy",
    class_weight='balanced',
    min_samples_leaf=5,
    min_samples_split=25)

In [ ]:
custom_cv_1 = custom_cv.split(xtrain, ytrain, groups)
fs = RFECV(clf_rfe,cv=custom_cv_1,scoring="f1_micro",step=0.1,verbose=2,n_jobs=4)
fs.fit(xtrain, ytrain)

In [ ]:
#support = fs.support_
#feature = pd.Series(xtrain.columns.values)
#selected_features = list(feature[support])
selected_features = ['Depth',
 'GR',
 'ILD_log10',
 'DeltaPHI',
 'PHIND',
 'PE',
 'NM_M',
 'RELPOS',
 'formation_size',
 'formation_depth',
 'soft_indic',
 'Depth_var',
 'GR_min',
 'GR_max',
 'GR_var',
 'ILD_log10_min',
 'ILD_log10_max',
 'ILD_log10_mean',
 'ILD_log10_var',
 'DeltaPHI_var',
 'PHIND_min',
 'PHIND_mean',
 'PE_min',
 'PE_max',
 'PE_mean',
 'NM_M_max',
 'NM_M_mean',
 'RELPOS_mean',
 'RELPOS_var',
 'Depth_min_dist',
 'Depth_max_dist',
 'Depth_mean_dist',
 'GR_min_dist',
 'GR_max_dist',
 'GR_mean_dist',
 'ILD_log10_min_dist',
 'ILD_log10_max_dist',
 'ILD_log10_mean_dist',
 'DeltaPHI_min_dist',
 'DeltaPHI_mean_dist',
 'PHIND_min_dist',
 'PHIND_max_dist',
 'PHIND_mean_dist',
 'RELPOS_min_dist',
 'RELPOS_max_dist',
 'RELPOS_mean_dist',
 'GR_lead_4',
 'GR_lead_5',
 'GR_lag_6',
 'GR_lead_6',
 'GR_lag_7',
 'GR_lead_7',
 'GR_lead_8',
 'GR_lead_9',
 'GR_lag_10',
 'GR_lead_10',
 'ILD_log10_lead_6',
 'ILD_log10_lead_7',
 'ILD_log10_lag_8',
 'ILD_log10_lead_8',
 'ILD_log10_lag_9',
 'ILD_log10_lead_9',
 'ILD_log10_lag_10',
 'ILD_log10_lead_10',
 'DeltaPHI_lead_9',
 'PHIND_lag_6',
 'PHIND_lead_6',
 'PHIND_lead_7',
 'PHIND_lead_8',
 'PHIND_lead_9',
 'PHIND_lag_10',
 'PHIND_lead_10',
 'RELPOS_lead_1',
 'RELPOS_lag_2',
 'RELPOS_lead_2',
 'RELPOS_lag_3',
 'RELPOS_lead_3',
 'RELPOS_lag_4',
 'RELPOS_lead_4',
 'RELPOS_lag_5',
 'RELPOS_lead_5',
 'RELPOS_lead_6',
 'RELPOS_lag_7',
 'RELPOS_lead_7',
 'RELPOS_lead_8',
 'RELPOS_lead_9',
 'RELPOS_lead_10',
 'Depth_rollingmean_5',
 'Depth_rollingmax_5',
 'Depth_rollingmin_5',
 'GR_rollingmean_5',
 'GR_rollingmax_5',
 'GR_rollingmin_5',
 'ILD_log10_rollingmean_5',
 'ILD_log10_rollingmax_5',
 'ILD_log10_rollingmin_5',
 'DeltaPHI_rollingmean_5',
 'DeltaPHI_rollingmax_5',
 'DeltaPHI_rollingmin_5',
 'PHIND_rollingmean_5',
 'PHIND_rollingmax_5',
 'PHIND_rollingmin_5',
 'PE_rollingmean_5',
 'PE_rollingmax_5',
 'PE_rollingmin_5',
 'NM_M_rollingmean_5',
 'NM_M_rollingmax_5',
 'NM_M_rollingmin_5',
 'RELPOS_rollingmean_5',
 'RELPOS_rollingmax_5',
 'RELPOS_rollingmin_5',
 'RELPOS_rollingstd_5',
 'Depth_rollingmean_10',
 'Depth_rollingmax_10',
 'Depth_rollingmin_10',
 'GR_rollingmean_10',
 'GR_rollingmax_10',
 'GR_rollingmin_10',
 'ILD_log10_rollingmean_10',
 'ILD_log10_rollingmax_10',
 'ILD_log10_rollingmin_10',
 'DeltaPHI_rollingmean_10',
 'DeltaPHI_rollingmax_10',
 'DeltaPHI_rollingmin_10',
 'DeltaPHI_rollingstd_10',
 'PHIND_rollingmean_10',
 'PHIND_rollingmax_10',
 'PHIND_rollingmin_10',
 'PE_rollingmean_10',
 'PE_rollingmax_10',
 'PE_rollingmin_10',
 'NM_M_rollingmean_10',
 'NM_M_rollingmin_10',
 'RELPOS_rollingmean_10',
 'RELPOS_rollingmax_10',
 'RELPOS_rollingmin_10',
 'RELPOS_rollingstd_10',
 'Depth_rollingmean_15',
 'Depth_rollingmax_15',
 'Depth_rollingmin_15',
 'GR_rollingmean_15',
 'GR_rollingmin_15',
 'ILD_log10_rollingmean_15',
 'ILD_log10_rollingmax_15',
 'ILD_log10_rollingmin_15',
 'ILD_log10_rollingstd_15',
 'DeltaPHI_rollingmean_15',
 'DeltaPHI_rollingmax_15',
 'DeltaPHI_rollingmin_15',
 'PHIND_rollingmean_15',
 'PHIND_rollingmax_15',
 'PHIND_rollingstd_15',
 'PE_rollingmean_15',
 'PE_rollingmax_15',
 'PE_rollingmin_15',
 'NM_M_rollingmean_15',
 'NM_M_rollingmax_15',
 'RELPOS_rollingmean_15',
 'RELPOS_rollingmax_15',
 'RELPOS_rollingmin_15',
 'RELPOS_rollingstd_15',
 'Depth_rollingmean_20',
 'Depth_rollingmax_20',
 'Depth_rollingmin_20',
 'GR_rollingmean_20',
 'GR_rollingmax_20',
 'GR_rollingmin_20',
 'ILD_log10_rollingmean_20',
 'ILD_log10_rollingmin_20',
 'ILD_log10_rollingstd_20',
 'DeltaPHI_rollingmean_20',
 'DeltaPHI_rollingmax_20',
 'DeltaPHI_rollingmin_20',
 'DeltaPHI_rollingstd_20',
 'PHIND_rollingmean_20',
 'PHIND_rollingmax_20',
 'PHIND_rollingstd_20',
 'PE_rollingmean_20',
 'PE_rollingmax_20',
 'PE_rollingmin_20',
 'PE_rollingstd_20',
 'NM_M_rollingmean_20',
 'RELPOS_rollingmean_20',
 'RELPOS_rollingmax_20',
 'RELPOS_rollingmin_20',
 'RELPOS_rollingstd_20',
 'Depth_rollingmean_50',
 'Depth_rollingmax_50',
 'Depth_rollingmin_50',
 'GR_rollingmean_50',
 'GR_rollingmin_50',
 'GR_rollingstd_50',
 'ILD_log10_rollingmean_50',
 'ILD_log10_rollingmin_50',
 'ILD_log10_rollingstd_50',
 'DeltaPHI_rollingmean_50',
 'DeltaPHI_rollingstd_50',
 'PHIND_rollingmean_50',
 'PHIND_rollingmax_50',
 'PHIND_rollingstd_50',
 'PE_rollingmean_50',
 'PE_rollingmax_50',
 'PE_rollingmin_50',
 'PE_rollingstd_50',
 'NM_M_rollingmean_50',
 'NM_M_rollingstd_50',
 'RELPOS_rollingmean_50',
 'RELPOS_rollingstd_50',
 'NM_M_Rank_Target_1_up',
 'NM_M_Rank_Target_2_up',
 'NM_M_Rank_Target_1_down',
 'NM_M_Rank_Target_2_down',
 u'lm_Formation',
 u'sh_Formation',
 u'lm_Formation_lag_1',
 u'sh_Formation_lag_1',
 u'lm_Formation_lead_1',
 u'sh_Formation_lead_1',
 u'lm_Formation_lag_2',
 u'sh_Formation_lag_2',
 u'lm_Formation_lead_2',
 u'sh_Formation_lead_2']
print(len(selected_features))
xtrain_fs = xtrain[selected_features]
xvalid_fs = xvalid[selected_features]

In [ ]:
rf = RandomForestClassifier(
            n_estimators=100,
            criterion="entropy",
            class_weight='balanced',
            min_samples_leaf=10,
            min_samples_split=20,
            max_features=50,
            max_depth= None,
            random_state=42
)

xtc =  ExtraTreesClassifier(
            n_estimators=100,
            criterion="entropy",
            class_weight='balanced',
            min_samples_leaf=10,
            min_samples_split=20,
            max_features=50,
            max_depth= None,
            random_state=42
)

gbt = GradientBoostingClassifier(
            loss='deviance',
            n_estimators = 100, 
            learning_rate = 0.1, 
            max_depth = 3,
            max_features = 10,
            min_samples_leaf = 5,
            min_samples_split = 25,
            random_state = 42, 
            max_leaf_nodes = None
)

xgb = XGBClassifier(
            learning_rate = 0.1, 
            max_depth = 3, 
            min_child_weight = 10, 
            n_estimators = 150, 
            colsample_bytree = 0.8,
            seed = 42
)

ensemble = VotingClassifier(
    estimators=[
            ('rf', rf), 
            ('xtc', xtc), 
            ('gbt', gbt), 
            ('xgb', xgb)
        ], 
    voting='soft',
    weights = [0.3,
               0.2,
              0.1,
              0.4],
    n_jobs=4
)

In [ ]:
ensemble.fit(xtrain_fs, ytrain)


In [ ]:
well_name_valid = df.loc[(df['origin']=='test'),"Well Name"]

In [ ]:
preds = ensemble.predict(xvalid_fs)

In [ ]:
well = "CRAWFORD"
depth = xvalid.loc[well_name_valid== well ,"Depth"]
predictions = pd.Series(preds).loc[well_name_valid==well]
plt.plot(depth,predictions)
plt.axis([2950,3175, 1, 9])
plt.grid(b=True, which='major', color='r', linestyle='--')
plt.show()

In [ ]:
well = "STUART"
depth = xvalid.loc[well_name_valid== well ,"Depth"]
predictions = pd.Series(preds).loc[well_name_valid==well]
plt.plot(depth,predictions)
plt.axis([2800,3050, 1, 9])
plt.grid(b=True, which='major', color='r', linestyle='--')
plt.show()

In [ ]:
xvalid['Facies']=preds
xvalid.to_csv('E1output.csv')